In [2]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [8]:
df = pd.read_csv("../data/adult.csv").replace("?", np.nan).dropna()
df

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
0,0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,39
1,1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35
2,2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,27
3,3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,43
4,4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,32556,27,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,38,United-States,<=50K,36
32557,32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,United-States,>50K,173
32558,32558,58,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,40,United-States,<=50K,40
32559,32559,22,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,20,United-States,<=50K,38


# Task 1
Print the count of men and women in the dataset.

In [9]:
men = df["sex"].tolist().count("Male")
women = df["sex"].tolist().count("Female")
print(f"Mens: {men}, Women's: {women}")

Mens: 20380, Women's: 9782


# Task 2
Find the average age of men in dataset

In [12]:
average_men_age = df['age'].where(df['sex'] == 'Male').mean()
average_men_age

39.18400392541707

# Task 3
Get the percentage of people from Poland (native-country)

In [155]:
from_poland = df.value_counts(df['native-country'] == 'Poland')[1] / df['native-country'].count() * 100
print("Poland people percentage: {0:.3f}".format(from_poland))

Poland people percentage: 0.186


# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [70]:
more_than_50k = df[df['salary'] == '>50K']['age']
less_than_50k = df[df['salary'] == '<50K']['age']
mean_more_than_50k = df[df['salary'] == '>50K']['age'].mean()
mean_less_than_50k = df[df['salary'] == '<=50K']['age'].mean()
more_sd = more_than_50k.std()
less_sd = less_than_50k.std()
print(f"The average age of people who earn >50K $ per year - {mean_more_than_50k}")
print(f"The average age of people who earn <=50K $ per year - {mean_less_than_50k}")
print(f"The age standard deviation of people who earn >50K $ per year - {more_sd}")
print(f"The age standard deviation of people who earn <=50K $ per year - {less_sd}")

The average age of people who earn >50K $ per year - 43.95911028236548
The average age of people who earn <=50K $ per year - 36.60806038668668
The age standard deviation of people who earn >50K $ per year - 10.269632835673852
The age standard deviation of people who earn <=50K $ per year - nan


# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [86]:
high_education = ["Bachelors", "Prof-school", "Assoc-acdm", "Assoc-voc", "Masters", "Doctorate"]
people_without_high_education_with_salary_more_than_50k = df[df['salary'] == ">50K"].where(~df['education'].isin
(high_education))
people_without_high_education_with_salary_more_than_50k

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
7,7.0,52.0,Self-emp-not-inc,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,45.0,United-States,>50K,307.0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10.0,37.0,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,Black,Male,80.0,United-States,>50K,116.0
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32557,32557.0,40.0,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40.0,United-States,>50K,173.0


# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [95]:
education_statistics = df.groupby('education')['age'].describe()
education_statistics

,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
10th,820.0,37.897561,16.225795,17.0,23.0,36.0,52.0,90.0
11th,1048.0,32.363550,15.089307,17.0,18.0,28.5,43.0,90.0
12th,377.0,32.013263,14.373710,17.0,19.0,28.0,41.0,79.0
1st-4th,151.0,44.622517,14.929051,19.0,33.0,44.0,56.0,81.0
5th-6th,288.0,41.649306,14.754622,17.0,28.0,41.0,53.0,82.0
7th-8th,557.0,47.631957,15.737479,17.0,34.0,49.0,60.0,90.0
9th,455.0,40.303297,15.335754,17.0,28.0,38.0,53.0,90.0
Assoc-acdm,1008.0,37.286706,10.509755,19.0,29.0,36.0,44.0,90.0
Assoc-voc,1307.0,38.246366,11.181253,19.0,30.0,37.0,45.0,84.0


# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [144]:
maried_mens_salaries = df[df['marital-status'].str.startswith("Married")].groupby('salary').count()
not_maried_mens_salaries = df[~df['marital-status'].str.startswith("Married")].groupby('salary').count()

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary K$
salary,,,,,,,,,,,,
<=50K,-8017,-8017,-8017,-8017,-8017,-8017,-8017,-8017,-8017,-8017,-8017,-8017
>50K,-6441,-6441,-6441,-6441,-6441,-6441,-6441,-6441,-6441,-6441,-6441,-6441


# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [150]:
max_hours_per_week = df['hours-per-week'].max()
people_with_max = df['hours-per-week'].value_counts()[max_hours_per_week]
people_with_max

78

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [157]:
df.corr()

C:\Users\Анастасия\AppData\Local\Temp\ipykernel_13440\1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,Unnamed: 0,age,hours-per-week,salary K$
Unnamed: 0,1.000000,-0.001126,-0.001890,0.000129
age,-0.001126,1.000000,0.101599,0.208203
hours-per-week,-0.001890,0.101599,1.000000,0.196378
salary K$,0.000129,0.208203,0.196378,1.000000
